# 유튜브 인기동영상 데이터

In [352]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/youtube.csv",index_col=0)
df.head()

,title,channelTitle,categoryId,view_count,likes,dislikes,comment_count,channelId,trending_date2
0,[신병] 물자창고,장삐쭈,23,1893473,38249,730,8595,UChbE5OZQ6dRHECsX0tEPEZQ,2021-01-01
1,RAIN(비) - 나로 바꾸자 Switch to me (duet with JYP) MV,RAIN's Official Channel,10,2600864,0,0,20129,UCxXgIeE5hxWxHG6dz9Scg2w,2021-01-01
2,2020년 제야의 종 온라인 타종행사 | 보신각 현장 행사는 진행하지 않습니다.,서울시 · Seoul,29,347049,3564,120,178,UCZUPZW5idAxYp-Asj__lVAA,2021-01-01
3,고기남자의 칠면조 파티,고기남자 MeatMan,26,528458,15372,280,3470,UCT3CumbFIJiW33uq0UI3zlg,2021-01-01
4,골목 3mc를 분노하게 만든 마음고생이 심했을 공릉 백반집 사장님의 푸념?! [예능...,스브스밥집,24,494904,3918,111,3142,UCdWgRSfttvDucq4ApcCg5Mw,2021-01-01


In [44]:
## 인기동영상 제작횟수가 많은 채널 상위 10개명을 출력하라 (날짜기준, 중복포함)
result = list(df.groupby('channelTitle').count().sort_values(by='trending_date2', ascending=False).head(10).index)

print(result)

['짤툰', '파뿌리', 'SPOTV', '런닝맨 - 스브스 공식 채널', '엠뚜루마뚜루 : MBC 공식 종합 채널', '장삐쭈', 'BANGTANTV', '채널 십오야', '이과장', '총몇명']


In [68]:
## 논란으로 인기동영상이 된 케이스를 확인하고 싶다. dislikes수가 like 수보다 높은 동영상을 제작한 채널을 모두 출력하라
channel_list = df[df['dislikes'] > df['likes']]['channelId'].unique()
result1 = list(df[df['channelId'].isin(channel_list)]['channelTitle'].unique())
print(result1)

result2 = list(df.loc[df['dislikes'] > df['likes']]['channelTitle'].unique())
print(result2)

['나혼자산다 STUDIO', '사나이 김기훈', '핫도그TV', 'ASMR 애정TV', '하얀트리HayanTree', '(MUTUBE)와꾸대장봉준', '양팡 YangPang', '철구형 (CHULTUBE)', '왜냐맨하우스', '오메킴TV', '육지담', 'MapleStory_KR', 'ROAD FIGHTING CHAMPIONSHIP', 'Gen.G esports']
['핫도그TV', 'ASMR 애정TV', '하얀트리HayanTree', '양팡 YangPang', '철구형 (CHULTUBE)', '왜냐맨하우스', '(MUTUBE)와꾸대장봉준', '오메킴TV', '육지담', 'MapleStory_KR', 'ROAD FIGHTING CHAMPIONSHIP', '사나이 김기훈', '나혼자산다 STUDIO', 'Gen.G esports']


In [127]:
## 채널명을 바꾼 케이스가 있는지 확인하고 싶다. channelId의 경우 고유값이므로 이를 통해 채널명을 한번이라도 바꾼 채널의 갯수를 구하여라
case = df[['channelTitle', 'channelId']].drop_duplicates().value_counts(['channelId'])
case = case[case >= 2]
result = len(case)
print(result)

71


In [245]:
## 일요일에 인기있었던 영상들중 가장많은 영상 종류(categoryId)는 무엇인가?
df['trending_date2'] = pd.to_datetime(df['trending_date2']) # datetime으로 형식 변환 
df1 = df[df['trending_date2'].dt.day_name() == 'Sunday'] # 일요일에 인기 있는 영상 목록
result = df1['categoryId'].value_counts().index[0] # 가장 많은 영상 종류
print(result)

24


In [321]:
## 각 요일별 인기 영상들의 categoryId는 각각 몇개 씩인지 하나의 데이터 프레임으로 표현하라
df['trending_date2'] = pd.to_datetime(df['trending_date2']) # object를 datetime으로 변환
df['trending_date2'] = df['trending_date2'].dt.day_name() # datetime을 요일로 변환
df1 = df.groupby(['categoryId', 'trending_date2'], as_index=False).size()
display(df1.pivot(index='categoryId',columns='trending_date2'))

size                                                  
trending_date2 Friday Monday Saturday Sunday Thursday Tuesday Wednesday
categoryId                                                             
1                 243    263      255    274      246     257       234
2                 120    105      119     99      128     119       129
10                833    837      776    830      890     894       917
15                187    215      198    217      207     208       207
17                633    668      592    636      682     708       706
19                 90     92       87     91       92      89        85
20                283    298      296    289      282     285       291
22               1288   1373     1289   1337     1341    1375      1333
23                568    594      570    556      560     569       566
24               2976   3148     3066   3096     2954    3084      3090
25                444    453      422    437      470     452       468
26                369    378      364    363      375     394       385
27                183    205      183    199      194     194       212
28                171    160      173    167      166     161       165
29                 12     10       10      9       13      11        12

In [340]:
## 댓글의 수로 (comment_count) 영상 반응에 대한 판단을 할 수 있다. viewcount대비 댓글수가 가장 높은 영상을 확인하라 (view_count값이 0인 경우는 제외한다)
df1 = df.loc[df['view_count'] != 0] # view_count==0인것 제외
df1['comment/view_ratio'] = df1['comment_count']/df1['view_count']
result = df1.loc[df1['comment/view_ratio'] == df1['comment/view_ratio'].max()]['title'].values
print(result)

['60분 동안 댓글이 달리지 않으면, 영상이 삭제됩니다. (챌린지)']


C:\Users\ccmedia\AppData\Local\Temp\ipykernel_9660\985911437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['comment/view_ratio'] = df1['comment_count']/df1['view_count']


In [350]:
## like 대비 dislike의 수가 가장 적은 영상은 무엇인가? (like, dislike 값이 0인경우는 제외한다)
df1 = df.loc[(df['likes']!=0) & (df['dislikes']!=0)] # likes, dislikes가 0인 것 제외
df1['dislikes/likes_ratio'] = df1['dislikes']/df1['likes']
result = df1.loc[df1['dislikes/likes_ratio'] == df1['dislikes/likes_ratio'].min()]['title'].values
print(result)

['[줌터뷰] *최초공개* 사부작즈🐰🐶의 비공식 이름은 아이라인즈? 꿀조합 티키타카 가득한 NCT 127 도영&정우의 줌터뷰']


C:\Users\ccmedia\AppData\Local\Temp\ipykernel_9660\2185969375.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['dislikes/likes_ratio'] = df1['dislikes']/df1['likes']


In [393]:
## 가장많은 트렌드 영상을 제작한 채널의 이름은 무엇인가?
result = df.loc[df['channelId']==df.value_counts('channelId').index[0]]['channelTitle'].values[0]
result 

'짤툰'

In [454]:
## 20회(20일)이상 인기동영상 리스트에 포함된 동영상의 숫자는?
result = (df[['title','channelId']].value_counts()>=20).sum()
print(result)

40
